## Read in data

In [1]:
import pandas as pd
import numpy as np

In [2]:
oo = pd.read_csv('../data/online_retail_small.csv')

In [3]:
d1 = oo.sample(n=4)

In [4]:
d2 = oo.sample(n=4)

## Testing column splitter

In [34]:
d2 = oo.sample(n=4)

In [39]:
d2

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
4382,536771,22966,GINGERBREAD MAN COOKIE CUTTER,12,2/12/2010 14:49,1.25,17581.0,United Kingdom
2613,536592,21899,"KEY FOB , GARAGE DESIGN",1,1/12/2010 17:06,1.66,NaN,United Kingdom
1210,536530,21664,RIDGED GLASS STORAGE JAR CREAM LID,2,1/12/2010 13:21,3.75,17905.0,United Kingdom
2427,536591,22386,JUMBO BAG PINK POLKADOT,1,1/12/2010 16:57,1.95,14606.0,United Kingdom


In [15]:
d2.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [68]:
def column_splitter(df, col_name):
    org_col_index = df.columns
    split_id = np.argwhere(org_col_index == col_name).reshape(1)[0]
    
    # split the original df into three sections: 
    # to_keep, col_to_split and cols_to_join_back
    to_keep = df.iloc[:, :split_id].copy(deep=True)
    to_split = df[[col_name]].copy(deep=True)
    to_join = df.iloc[:, (split_id+1):].copy(deep=True)
    cols_to_add = np.hstack((org_col_index[(split_id+1):], ''))
    
    #split the column:
    to_split = to_split[col_name].str.split("(?=,)",n=1, expand=True)
    to_split.columns = [col_name, cols_to_add[0]]
    
    # join the split column back first
    #to_keep = pd.concat([to_keep, to_split], axis=1)
    to_keep = to_keep.join(to_split)
    
    return to_keep

In [69]:
column_splitter(d2, 'Description')

,InvoiceNo,StockCode,Description,Quantity
4382,536771,22966,GINGERBREAD MAN COOKIE CUTTER,None
2613,536592,21899,KEY FOB,", GARAGE DESIGN"
1210,536530,21664,RIDGED GLASS STORAGE JAR CREAM LID,None
2427,536591,22386,JUMBO BAG PINK POLKADOT,None


In [65]:
pd.DataFrame.join?

Signature:
pd.DataFrame.join(
    self,
    other,
    on=None,
    how='left',
    lsuffix='',
    rsuffix='',
    sort=False,
) -> 'DataFrame'
Docstring:
Join columns of another DataFrame.

Join columns with `other` DataFrame either on index or on a key
column. Efficiently join multiple DataFrame objects by index at once by
passing a list.

Parameters
----------
other : DataFrame, Series, or list of DataFrame
    Index should be similar to one of the columns in this one. If a
    Series is passed, its name attribute must be set, and that will be
    used as the column name in the resulting joined DataFrame.
on : str, list of str, or array-like, optional
    Column or index level name(s) in the caller to join on the index
    in `other`, otherwise joins index-on-index. If multiple
    values given, the `other` DataFrame must have a MultiIndex. Can
    pass an array as the join key if it is not already contained in
    the calling DataFrame. Like an Excel VLOOKUP operation.
how : {'lef

In [30]:
id = pd.Series([True, False, False, True])

In [31]:
id.index = qty_series.index

In [32]:
qty_series[id] = d2.Description[id]

In [33]:
qty_series

3159    FULL ENGLISH BREAKFAST PLATE
3348                            12.0
4596                             2.0
4405     ZINC METAL HEART DECORATION
Name: Quantity, dtype: object

In [24]:
d2a = d2.Description.str.split('-',n=1, expand=True)

In [25]:
d2a[[1]]

,1
4608,None
1675,None
4451,None
1762,LIGHT HLDR


In [26]:
d2[["Quantity"]]

,Quantity
4608,1
1675,1
4451,108
1762,4


In [28]:
d2.combine_first?

Signature: d2.combine_first(other: 'DataFrame') -> 'DataFrame'
Docstring:
Update null elements with value in the same location in `other`.

Combine two DataFrame objects by filling null values in one DataFrame
with non-null values from other DataFrame. The row and column indexes
of the resulting DataFrame will be the union of the two.

Parameters
----------
other : DataFrame
    Provided DataFrame to use to fill null values.

Returns
-------
DataFrame

See Also
--------
DataFrame.combine : Perform series-wise operation on two DataFrames
    using a given function.

Examples
--------
>>> df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
>>> df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
>>> df1.combine_first(df2)
     A    B
0  1.0  3.0
1  0.0  4.0

Null values still persist if the location of that null value
does not exist in `other`

>>> df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
>>> df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
>>> df1.combine_first(df2)
 

In [27]:
pd.concat([d2a[[1]],d2[["Quantity"]]], axis=1)

,1,Quantity
4608,None,1
1675,None,1
4451,None,108
1762,LIGHT HLDR,4


In [65]:
oo.Description.str.contains("[^\w\s]", regex=True).isna().sum()

12

In [68]:
oo2 = oo[~oo.Description.isna()]

In [73]:
tmp = oo2[oo2.Description.str.contains("[^\w\s]", regex=True)].Description

In [77]:
tmp.str.split("(?=-)",n=1, expand=True)

,0,1
0,WHITE HANGING HEART T,-LIGHT HOLDER
4,RED WOOLLY HOTTIE WHITE HEART.,None
6,GLASS STAR FROSTED T,-LIGHT HOLDER
10,POPPY'S PLAYHOUSE BEDROOM,None
11,POPPY'S PLAYHOUSE KITCHEN,None
...,...,...
4969,SET OF 6 T,-LIGHTS SNOWMEN
4982,DINOSAUR PARTY BAG + STICKER SET,None
4986,EUCALYPTUS & PINECONE WREATH,None
4988,"SWISS ROLL TOWEL, CHOCOLATE SPOTS",None
